<a href="https://colab.research.google.com/github/limjustin/Do_it_Deep_Learning/blob/master/Chapter08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 08-1. 합성곱 연산에 대해 알아보기

#### 합성곱(convolution) 연산
**개념**
- 원본 배열 x와 미끄러지는 배열 w가 존재
- w 배열의 원소 순서를 뒤집음(reverse)
- x와 w의 각 배열 원소끼리 곱한 후 더하는 연산을 수행 (점 곱 연산)
- w를 오른쪽으로 한 칸 이동시켜서 같은 연산 수행

**구현**

In [ ]:
## 1. 넘파이 배열 정의하고 배열 하나 선택해 뒤집기

import numpy as np
w = np.array([2, 1, 5, 3])
x = np.array([2, 8, 3, 7, 1, 2, 0, 4, 5])

w_r = np.flip(w)    # flip() 함수를 사용해 배열에 대한 reverse 연산 수행
print(w_r)

[3 5 1 2]


In [ ]:
## 2. 넘파이의 점 곱으로 합성곱 연산 수행하기

for i in range(6):
  print(np.dot(x[i:i+4], w_r))

63
48
49
28
21
20


In [ ]:
## 3. 싸이파이로 합성곱 수행하기

from scipy.signal import convolve
convolve(x, w, mode='valid')    # scipy에서 합성곱 연산을 위한 함수 convolve()

array([63, 48, 49, 28, 21, 20])

#### 교차 상관(cross-correlation) 연산
**개념**
- 사실 합성곱 신경망은 교차 상관을 사용
- 합성곱과 동일한 방법으로 연산을 진행
- 하지만 **'미끄러지는 배열을 뒤집지 않는다'**는 점이 다름

**구현**

In [ ]:
from scipy.signal import correlate
correlate(x, w, mode='valid')

array([48, 57, 24, 25, 16, 39])

**합성곱 신경망에서 교차 상관을 사용하는 이유**
```
모델 훈련 과정
1. 가중치를 무작위 값으로 초기화
2. 모든 샘플에 대하여 정방향과 역방향 계산을 수행하여 가중치를 조금씩 학습(업데이트)
```
- 모든 모델은 훈련하기 전에 가중치 배열의 요소들을 무작위로 초기화함
- 여기서 가중치 배열은 '미끄러지는 배열'
- 가중치 배열은 무작위로 이미 초기화되어 있기 때문에 **가중치를 뒤집어서 적용하던지 여부는 상관이 없음**
- 그래서 그냥 **가중치 배열을 뒤집지 않는 교차 상관**을 사용한다!!

#### 패딩(padding)
**개념**
- **원본 배열**의 양 끝에 빈 원소를 추가하는 것

**종류**
- 밸리드 패딩(valid padding)
  - 밸리드 패딩의 결과로 얻는 배열의 크기는 항상 원본 배열의 크기보다 작음
  - 원본 배열의 각 원소가 **연산에 참여하는 정도가 다른 것**이 특징
  -```correlate(x, w, mode='valid')```
- 풀 패딩(full padding)
  - **원본 배열의 양 끝에 가상의 원소를 추가**하는 '제로 패딩(zero padding)' 과정이 필요
  - 적절한 개수의 제로 패딩을 추가하면 **원본 배열의 모든 원소가 연산에 동일하게 참여**할 수 있음
  - **원본 배열의 모든 요소가 동일하게 연산에 참여**하는 것이 특징
  -```correlate(x, w, mode='full')```  
- 세임 패딩(same padding)
  - **출력 배열의 길이가 원본 배열의 길이와 같아지도록** 원본 배열에 제로 패딩을 추가
  - ```correlate(x, w, mode='same')```

#### 스트라이드(stride)
**개념**
- 미끄러지는 배열의 간격을 조절하는 것
- 합성곱 신경망을 만들 때는 보통 스트라이드를 1로 지정

#### 2차원 배열에서 합성곱 수행하기
- 합성곱의 수행 방향 : 왼쪽 -> 오른쪽, 위 -> 아래

In [ ]:
x = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
w = np.array([[2, 0], [0, 0]])
from scipy.signal import correlate2d    # correlate2d() 함수를 사용하여 2차원 배열의 합성곱 계산
correlate2d(x, w, mode='valid')
correlate2d(x, w, mode='same')

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]])

#### 텐서플로로 합성곱 수행하기
**원리**
- scipy 말고 tensorflow에서도 합성곱을 위한 함수 제공
- 원본 배열 : 입력 / 미끄러지는 배열 : 가중치

**합성곱 신경망의 입력은 일반적으로 4차원 배열**
- 입력으로 4차원 배열을 기대함
- 이유 : 입력 이미지의 **높이와 너비 외에 더 많은 차원이 필요**하기 때문
- 입력 배열의 구성
  - (배치, 샘플의 높이, 샘플의 너비, 컬러 채널의 차원)
- 가중치 배열의 구성
  - (가중치의 높이, 가중치의 너비, 채널, 가중치의 개수)
- 합성곱 배열의 구성
  - (입력의 배치, 입력의 높이, 입력의 너비, 필터의 개수)

**2차원 배열을 4차원 배열로 바꿔 합성곱 수행**

In [ ]:
import tensorflow as tf
x_4d = x.astype(np.float).reshape(1, 3, 3, 1)   # reshape() 메서드로 2차원 배열에서 4차원 배열로 바꾸기
w_4d = w.reshape(2, 2, 1, 1)                    # astype() 메서드로 자료형을 실수로 바꾸기

c_out = tf.nn.conv2d(x_4d, w_4d, strides=1, padding='SAME')   # 텐서플로에서 2차원 합성곱을 수행하는 함수는 conv2d() / 매개변수는 4차원 배열!!

c_out.numpy().reshape(3, 3)

array([[ 2.,  4.,  6.],
       [ 8., 10., 12.],
       [14., 16., 18.]])

#### 합성곱 신경망이 이미지 분류에 더 뛰어난 성능을 보이는 이유
- 가중치 배열의 크기가 작아짐(훨씬 가중치가 적음)
- 간단한 용어 정리
  - 필터(filter) : 합성곱의 가중치
  - 커널(kernel) : 케라스에서 합성곱의 **필터 1개**
  - '가중치'는 **필터 전체**를 지칭할 때 사용함


## 08-2. 풀링 연산에 대해 알아보기
**개념**
- 합성곱층 : 합성곱이 일어나는 층
- 풀링층 : 풀링이 일어나는 층
- 특성 맵(feature map) : 합성곱층과 풀링층에서 만들어진 결과
- 합성곱 신경망의 모습 : 합성곱층 뒤에 풀링층이 뒤따르는 형태
- 풀링 : 특성 맵을 스캔하며 최댓값을 고르거나 평균값을 계산하는 것

#### 최대 풀링(max pooling)
**개념**
- 특성 맵 위를 스캔하며 최댓값을 고르는 방식
- 풀링 영역의 크기는 보통 2*2를 지정
- 스트라이드는 풀링의 한 모서리 크기로 지정 (풀링 영역이 겹쳐지지 않도록 스캔)

**효과**
- 2*2 풀링은 특성 맵의 크기를 절반으로 줄임
- 특성 맵의 한 요소가 입력의 더 넓은 영역을 바라볼 수 있는 효과 有
- 최대 풀링의 요소들은 2*2 크기의 각 영역을 대표함

#### 평균 풀링(average pooling)
**개념**
- 풀링 영역의 평균값을 계산

#### 최대 풀링을 더 선호하는 이유
- 평균 풀링은 **합성곱층을 통과하는 특징들을 희석**시킬 가능성이 높음
- 입력에서 합성곱 필터가 찾고자 하는 부분은 **특성 맵의 가장 큰 값으로 활성화**
- 평균 풀링은 **가장 큰 특성의 값을 상쇄** <-> 최대 풀링은 **가장 큰 특징을 유지**
- 따라서, 최대 풀링이 **이미지 분류 작업에 더 적합**

In [ ]:
x = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 16]])
x = x.reshape(1, 4, 4, 1)

p_out = tf.nn.max_pool2d(x, ksize=2, strides=2, padding='VALID')
p_out.numpy().reshape(2, 2)

array([[ 6.,  8.],
       [14., 16.]], dtype=float32)

**풀링에 대해 필요한 사항**
- 풀링층에는 학습되는 가중치가 없음
- 풀링층을 통과하기 전후로 배치 크기와 채널 크기는 동일함

## 08-3. 합성곱 신경망의 구조 알아보기

#### 렐루(ReLU) 함수
**개념**
- 합성곱 신경망에서 자주 사용하는 활성화 함수
- 합성곱층에 적용되는 활성화 함수
- 합성곱 신경망의 성능을 더 높여줌
- x가 0보다 작으면 y=0, 0보다 크면 y=x
- 도함수는 x가 0보다 작으면 0, 0보다 크면 1

**구현**
```
def relu(x):
    return np.maximun(x, 0)
```
```
r_out = tf.nn.relu(x)  # tensorflow에서 제공하는 렐루 함수는 relu()
r_out.numpy()  # 화면에 출력하려면 Tensor 객체를 numpy로 변환해야 함
```

#### 합성곱 신경망에서 일어나는 일들과 구조
**1. 합성곱층에서 일어나는 일**
- 입력 데이터(이미지)에는 채널(channel)이라는 **차원을 하나 더 가짐**
- 핵심1 : 이미지의 **모든 채널에 합성곱이 한 번에 적용**되어야 함
  - 커널의 **마지막 차원**은 **입력 채널의 개수와 동일**해야 함
  - ex) 입력 채널이 4x4x10의 형태라면, 커널 배열의 크기도 3x3**x10**으로 마지막 차원의 개수를 동일하게 맞춰주어야 함
- 핵심2 : 입력 채널은 커널의 채널과 **각각 합성곱을 수행**, 그 후 합성곱의 전체 결과를 더하여 **특성 맵 1조각을 만듦**
  - **커널이 하나**면 이미지의 **특징을 하나만 감지**
  - **복수 개의 커널의 경우**에는 이미지에서 **여러 개의 특징을 감지**할 수 있음
- **정리 : 합성곱층에 주입된 이미지는 특성을 감지하는 커널에 의해 특성 맵으로 만들어짐**

**2. 풀링층에서 일어나는 일**
- 합성곱층을 통해 특성 맵이 만들어짐
- 핵심1 : 활성화 함수로 **렐루 함수가 적용**됨
- 핵심2 : **풀링** 적용
  - 풀링은 **특성 맵의 크기를 줄여줌**
  - BUT 채널의 크기는 줄어들지 않음
- **정리 : 합성곱층을 통해 만들어진 특성 맵은 활성화 함수(렐루)와 풀링층을 거쳐 더 작은 특성 맵이 됨 (채널의 개수는 유지)**

**3. 완전 연결층에서 일어나는 일**
- **합성곱층과 풀링층을 통과시켜 얻은 특성 맵**
- 1. 이는 **일렬**로 펼쳐 **완전 연결층에 입력으로 주입**
- 2. 출력층과 다중 분류를 위한 소프트맥스 함수를 통과하여 **최종 출력**을 만듦
- 완전 연결층의 출력은 **출력층의 뉴런**과 연결
- **정리 : 풀링층을 통해 만들어진 특성 맵을 일렬로 펼쳐 완전 연결층에 주입. 그 다음 출력층을 통과하여 최종 예측을 만듦**

## 08-4. 합성곱 신경망을 만들고 훈련하기



In [ ]:
class ConvolutionNetwork:

  def __init__(self, n_kernels=10, units=10, batch_size=32, learning_rate=0.1):
    self.n_kernels = n_kernels    # 합성곱의 커널 개수
    self.kernel_size = 3    # 커널 크기
    self.optimizer = None   # 옵티마이저
    self.conv_w = None    # 합성곱층의 가중치
    self.conv_b = None    # 합성곱층의 절편
    self.units = units    # 은닉층의 뉴런 개수
    self.batch_size = batch_size    # 배치 크기
    self.w1 = None    # 은닉층의 가중치
    self.b1 = None    # 은닉층의 절편
    self.w2 = None    # 출력층의 가중치
    self.b2 = None    # 출력층의 절편
    self.a1 = None    # 은닉층의 활성화 출력
    self.losses = []    # 훈련 손실
    self.val_losses = []    # 검증 손실
    self.lr = learning_rate   # 학습률

  def forpass(self, x):
    # 3x3 합성곱 연산을 수행합니다.
    c_out = tf.nn.conv2d(x, self.conv_w, strides=1, padding='SAME') + self.conv_b
    # 렐루 활성화 함수를 적용합니다.
    r_out = tf.nn.relu(c_out)
    # 2x2 최대 풀링을 적용합니다.
    p_out = tf.nn.max_pool2d(r_out, ksize=2, strides=2, padding='VALID')
    # 첫 번째 배치 차원을 제외하고 출력을 일렬로 펼칩니다.
    f_out = tf.reshape(p_out, [x.shape[0], -1])
    z1 = tf.matmul(f_out, self.w1) + self.b1   # 첫 번째 층의 선형식 계산
    a1 = tf.nn.relu(z1)    # 활성화 함수 적용
    z2 = tf.matmul(a1, self.w2) + self.b2   # 두 번째 층의 선형식 계산
    return z2

  def init_weights(self, input_shape, n_classes):
    g = tf.initializers.glorot_uniform()
    self.conv_w = tf.Variable(g((3, 3, 1, self.n_kernels)))
    self.conv_b = tf.Variable(np.zeros(self.n_kernels), dtype=float)
    n_features = 14 * 14 * self.n_kernels
    self.w1 = tf.Variable(g((n_features, self.units)))    # (특성 개수, 은닉층의 크기)
    self.b1 = tf.Variable(np.zeros(self.units), dtype=float)    # 은닉층의 크기
    self.w2 = tf.Variable(g((self.units, n_classes)))   # (은닉층의 크기, 클래스 개수)
    self.b2 = tf.Variable(np.zeros(n_classes), dtype=float)   # 클래스 개수

  def fit(self, x, y, epochs=100, x_val=None, y_val=None):
    self.init_weights(x.shape, y.shape[1])   # 은닉층과 출력층의 가중치 초기화
    self.optimizer = tf.optimizers.SGD(learning_rate=self.lr)
    # epochs 만큼 반복
    for i in range(epochs):
      print('에포크', i, end=' ')
      # 제너레이터 함수에서 반환한 미니 배치를 순환
      batch_losses = []
      for x_batch, y_batch in self.gen_batch(x, y):
          print('.', end='')
          self.training(x_batch, y_batch)
          # 배치 손실을 기록
          batch_losses.append(self.get_loss(x_batch, y_batch))
      print( )
      # 배치 손실 평균을 내어 훈련 손실값으로 저장
      self.losses.append(np.mean(batch_losses))
      # 검증 세트에 대한 손실 계산
      self.val_losses.append(self.get_loss(x_val, y_val))

  # 미니 배치 제너레이터 함수
  def gen_batch(self, x, y):
    bins = len(x) // self.batch_size   # 미니 배치 횟수
    indexes = np.random.permutation(np.arange(len(x)))    # 인덱스 섞기
    x = x[indexes]
    y = y[indexes]
    for i in range(bins):
      start = self.batch_size * i
      end = self.batch_size * (i + 1)
      yield x[start:end], y[start:end]    # batch_size만큼 슬라이싱하여 반환

  def training(self, x, y):
    m = len(x)    # 샘플 개수 저장
    with tf.GradientTape( ) as tape:
      z = self.forpass(x)   # 정방향 계산을 수행
      # 손실을 계산
      loss = tf.nn.softmax_cross_entropy_with_logits(y, z)
      loss = tf.reduce_mean(loss)
    
    weights_list = [self.conv_w, self.conv_b,
                    self.w1, self.b1, self.w2, self.b2]
    # 가중치에 대한 그레이디언트 계산
    grads = tape.gradient(loss, weights_list)
    # 가중치를 업데이트 함
    self.optimizer.apply_gradients(zip(grads, weights_list))

  def predict(self, x):
    z = self.forpass(x)   # 정방향 계산 수행
    return np.argmax(z.numpy(), axis=1)   # 가장 큰 값의 인덱스 반환

  def score(self, x, y):
    # 예측과 타깃 열 벡터를 비교하여 True의 비율을 반환
    return np.mean(self.predict(x) == np.argmax(y, axis=1))

  def get_loss(self, x, y):
    z = self.forpass(x)   # 정방향 계산 수행
    # 손실을 계산하여 저장
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, z))
    return loss.numpy()


#### 합성곱 신경망 훈련하기

In [ ]:
## 1. 데이터 세트 불러오기

(x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
## 2. 훈련 데이터 세트를 훈련 세트와 검증 세트로 나누기

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all,
                                                  stratify=y_train_all, test_size=0.2,
                                                  random_state=42)

In [ ]:
## 3. 타깃을 원-핫 인코딩으로 변환하기

y_train_encoded = tf.keras.utils.to_categorical(y_train)
y_val_encoded = tf.keras.utils.to_categorical(y_val)

In [ ]:
## 4. 입력 데이터 준비하기

x_train = x_train.reshape(-1, 28, 28, 1)      # 마지막에 명암을 나타내는 1차원 채널을 추가
x_val = x_val.reshape(-1, 28, 28, 1)

x_train.shape

(48000, 28, 28, 1)

In [ ]:
## 5. 입력 데이터 표준화 전처리하기

x_train = x_train / 255   # 0~255 사이의 정수로 픽셀 강도를 표현하기 위해
x_val = x_val / 255   # 255로 나누어 0~1 사이의 값으로 조정

In [ ]:
## 6. 모델 훈련하기

cn = ConvolutionNetwork(n_kernels=10, units=100, batch_size=128, learning_rate=0.01)
cn.fit(x_train, y_train_encoded, x_val=x_val, y_val=y_val_encoded, epochs=20)

에포크 0 ............................................................................................

KeyboardInterrupt: ignored